In [1]:
import os, sys, json

autocog_home = os.path.realpath(f"{os.getcwd()}/../")
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.utility.models import loader
from autocog.utility.pynb import wrap_graphviz, display

In [2]:
model_names = [
    'llama-2-7b.Q4_K_M',                       # wget https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_K_M.gguf
    'llama-2-7b-chat.Q4_K_M',                  # wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf
    'llama-2-13b-chat.Q4_K_M',                 # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_K_M.gguf
    'llama-2-13b-chat.Q5_K_M',                 # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf
    'llama-2-13b-chat.Q8_0',                   # wget https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q8_0.gguf
    'tinyllama-1.1b-chat-v0.3.Q4_K_M',         # wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf
    'tinyllama-2-1b-miniguanaco.Q4_K_M',       # wget https://huggingface.co/TheBloke/Tinyllama-2-1b-miniguanaco-GGUF/resolve/main/tinyllama-2-1b-miniguanaco.Q4_K_M.gguf
    'capybarahermes-2.5-mistral-7b.Q4_K_M'     # wget https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf
]
(lm,syntax) = loader(
    models_path=f"/data/models/{model_names[1]}.gguf",
    prompt_with_format=False,
    prompt_with_index=False,
    prompt_indent='  '
)

In [3]:
arch = CogArch(lm=lm, syntax=syntax)

mmlu_cogs = [
    arch.load('mcq.repeat.base',  filepath=f"@mcq/repeat.sta"),
    arch.load('mcq.repeat.cot',   filepath=f"@mcq/repeat-cot.sta"),
    arch.load('mcq.repeat.hyp',   filepath=f"@mcq/repeat-hyp.sta"),
    arch.load('mcq.repeat.iter',  filepath=f"@mcq/repeat-iter.sta"),
    arch.load('mcq.repeat.annot', filepath=f"@mcq/repeat-annot.sta"),
    arch.load('mcq.select.base',  filepath=f"@mcq/select.sta"),
    arch.load('mcq.select.cot',   filepath=f"@mcq/select-cot.sta"),
    arch.load('mcq.select.hyp',   filepath=f"@mcq/select-hyp.sta"),
    arch.load('mcq.select.iter',  filepath=f"@mcq/select-iter.sta"),
    arch.load('mcq.select.annot', filepath=f"@mcq/select-annot.sta")
]

mmlu_data = [
    {
        "topic"    : "arithmetic",
        "question" : "What is 3*4+9?",
        "choices"  : [ "16", "21", "39", "42" ]
    }
]

arith_cogs = [
    arch.load('elementary.multiply.single', filepath=f"@elementary/multiply-single.sta"),
    arch.load('elementary.multiply.chain',  filepath=f"@elementary/multiply-chain.sta")
]

arith_data = [
    {
        "problem" : "What is the product of 392 and 42?"
    }
]

In [4]:
# TODO node prefixed by prompt name
# dotstr = '\n'.join([ prompt.toGraphViz_concrete() for prompt in arith_cogs[0].prompts.values() ])
# dotstr = arith_cogs[0].prompts['main'].instantiate(syntax=arch.syntax, stacks={}, branches={}, inputs=arith_data[0]).toGraphViz()
# display(wrap_graphviz(dotstr))

In [5]:
tags = [
    'mcq.repeat.base', 'mcq.repeat.cot', 'mcq.repeat.hyp', 'mcq.repeat.iter', 'mcq.repeat.annot',
    'mcq.select.base', 'mcq.select.cot', 'mcq.select.hyp', 'mcq.select.iter', 'mcq.select.annot'
]
# tags = [ 'mmlu-repeat-cot', 'mmlu-select-cot' ]
results = [ await arch(tag, **data) for tag in tags for data in mmlu_data ]
# arith_results = [ await cog(**data) for cog in arith_cogs for data in arith_data ]
results

['42', '21', '42', '16', '42', 2, 2, 2, 2, 2]

In [6]:
# TODO inspect function...
import numpy
scoring = lambda probas: numpy.power(numpy.prod(probas), 1./len(probas))
texts = arch.orchestrator.pages[-8].ftts['main'][-1].results(lm)
for text in texts[-2:]:
    print(f"p={text[1]}\n---------------------\n\n{text[0]}\n\n============================\n")

p=0.6638551711390864
---------------------

[INST] <<SYS>>
You are an AI expert interacting with your environment using a set of interactive questionnaires.
<</SYS>>
You are answering a multiple choice questionnaire.
You are using the following syntax:
```
start:
  topic(text(20)): the general category from which the question was taken
  question(text(50)): the question that you have to answer
  choices(text(40))[4]: the four possible choices to answer the question, only one is correct
  work(thought)[1:10]: show your work step-by-step
  answer(select(choices)): you pick the index of the choice that best answer the question
next: select which of return will be the next step.
```
It includes the folowing named formats:
- thought: text(20)
  - a short text representing a single thought, it does not have to be a proper sentence.
[/INST]
start:
   topic: arithmetic
   question: What is 3*4+9?
   choices: 16
   choices: 21
   choices: 39
   choices: 42
   work:
   work: 3*4 = 12
   work: 12

In [7]:
# arch.orchestrator.pages[-1].ftts['main'][-1].toGraphViz(lm)